In [1]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue
import pyaudio, wave

In [2]:
message = Queue()
recording = Queue()

record_button = widgets.Button(
    description="Record",
    disabled = False,
    button_style="success", 
    icon  = 'microphone'
)

stop_button = widgets.Button(
    description="Stop",
    disabled = False,
    button_style="warning",
    icon  = "stop"
)

output = widgets.Output()

In [3]:
CHANNELS = 1
FRAME_RATE = 44100
RECORD_SECONDS = 10
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

In [4]:
def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format=AUDIO_FORMAT,
                    channels=CHANNELS,
                    rate = FRAME_RATE,
                    input = True,
                    input_device_index = 1,
                    frames_per_buffer=chunk)

    frames = []
    while not message.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frames) >= (FRAME_RATE / chunk) * RECORD_SECONDS:
            recording.put(frames.copy())
            frames = []
        
    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open('output.wav', 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(AUDIO_FORMAT))
    wf.setframerate(FRAME_RATE)
    wf.writeframes(b''.join(frames))
    wf.close() 

In [5]:
def start_record(data):
    message.put(True)

    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()


def stop_record(data):
    with output:
        message.get()
        display("Stopped")

record_button.on_click(start_record)
stop_button.on_click(stop_record)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()